In [2]:
## Basic import and DF creation

import pandas as pd

# Specify the path to your Excel file
file_path = r"09_Outputs\main_data.xlsx"

# Load the Excel file into a DataFrame using the openpyxl engine
df_main = pd.read_excel(file_path, engine='openpyxl')

# Remove quotation marks from the 'Entity' column
df_main['Entity'] = df_main['Entity'].str.replace('"', '')

# Retrieve unique countries from the DataFrame and store them as a list of strings
all_countries = df_main['Entity'].drop_duplicates().astype(str).tolist()

# Display the first 5 rows of the DataFrame for a quick overview

# Assuming df_main is your dataframe

min_year = df_main["Year"].min()
max_year = df_main["Year"].max()

# ... Rest of your code ...

df_main.head()

,Entity,Year,Code,Annual CO₂ emissions,Renewables (TWh growth - equivalent),Annual CO₂ emissions from flaring,Annual CO₂ emissions from gas,Annual CO₂ emissions from oil,Annual CO₂ emissions from coal,Primary energy consumption (TWh),...,Electricity from fossil fuels (TWh),Other renewables excluding bioenergy (TWh),Oil consumption - TWh,Primary energy consumption per GDP (kWh/$),Low-carbon electricity (TWh),GDP (constant 2015 US$),Renewable electricity (% electricity production) (World Bank (2015)),Solar photovoltaic capacity,Annual greenhouse gas emissions in CO2 equivalents,Electricity from wind (TWh)
0,Afghanistan,1965,AFG,1006917.0,NaN,0.0,0.0,542272.0,381056.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24430504.0,NaN
1,Afghanistan,1966,AFG,1091159.0,NaN,0.0,0.0,575248.0,428688.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25174948.0,NaN
2,Afghanistan,1967,AFG,1281865.0,NaN,0.0,260144.0,556928.0,399376.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24644464.0,NaN
3,Afghanistan,1968,AFG,1223391.0,NaN,0.0,347041.0,496817.0,332429.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24382198.0,NaN
4,Afghanistan,1969,AFG,941232.0,NaN,0.0,0.0,527616.0,362736.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23233260.0,NaN


In [6]:
##################### Section 1: Imports and Data Assumptions #####################
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming df_main is already loaded
filtered_df = df_main

##################### Section 2: Helper Functions #####################
all_countries = sorted(df_main['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']

def custom_sort(country):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    if country in default_countries:
        return (-2, default_countries.index(country))
    elif country in selected_countries:
        return (-1, selected_countries.index(country))
    else:
        return (1, country)

def update_country_list(*args):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    search_input = search_box.value
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    updated_countries = list(set(search_results + selected_countries))
    
    updated_countries = sorted(updated_countries, key=custom_sort)
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    for cb in updated_checkboxes:
        cb.observe(plot_selected_column, names='value')
        cb.observe(update_country_list, names='value')
    checkboxes_container.children = tuple(updated_checkboxes)

def plot_selected_column(change=None):
    column = column_dropdown.value
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    with out:
        clear_output(wait=True)
        fig = go.Figure()
        for country in selected_countries:
            country_data = filtered_df[filtered_df['Entity'] == country]
            fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
        
        fig.update_layout(title=column, 
                          xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          yaxis=dict(title=column, showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          plot_bgcolor='white', paper_bgcolor='white')
        fig.show()

def clear_search(*args):
    search_box.value = ''

##################### Section 3: Create Widgets #####################
checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country, layout=widgets.Layout(width="100%", padding="0px")) for country in sorted(all_countries, key=custom_sort)]
for cb in country_checkboxes:
    cb.observe(plot_selected_column, names='value')
    cb.observe(update_country_list, names='value')

checkboxes_container.children = tuple(country_checkboxes)

search_box = widgets.Text(value='', placeholder='Search for Countries...')
search_box.observe(update_country_list, names='value')

# Create the clear button and set its behavior
clear_button = widgets.Button(description="X")
clear_button.on_click(clear_search)



# Group the search box and clear button together
search_widget = widgets.HBox([search_box, clear_button])

out = widgets.Output(layout=widgets.Layout(width="60%"))
columns_to_plot = df_main.columns[df_main.columns.get_loc('Code')+1:]
column_dropdown = widgets.Dropdown(options=columns_to_plot, description='Select Column:')
column_dropdown.observe(plot_selected_column, names='value')

# Adjust the width of the dropdown
column_dropdown.layout.width = '30%'

checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))
country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country, layout=widgets.Layout(width="100%", padding="0px")) for country in sorted(all_countries, key=custom_sort)]
for cb in country_checkboxes:
    cb.observe(plot_selected_column, names='value')
    cb.observe(update_country_list, names='value')
checkboxes_container.children = tuple(country_checkboxes)

search_box = widgets.Text(value='', placeholder='Search for Countries...')
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X")
clear_button.on_click(clear_search)
search_widget = widgets.HBox([search_box, clear_button])

out = widgets.Output(layout=widgets.Layout(width="60%"))

# Displaying widgets
left_container = widgets.VBox([search_widget, checkboxes_container])
display(column_dropdown)
display(widgets.HBox([left_container, out]))
plot_selected_column()
update_country_list()  # initialize the country list

Dropdown(description='Select Column:', layout=Layout(width='30%'), options=('Annual CO₂ emissions', 'Renewable…

In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Assuming df_main is your dataframe
df_main = df_main[~df_main['Entity'].isin(['World', 'Continent'])]

app = dash.Dash(__name__)

# Create a dropdown list with available columns (starting from the 4th column)
dropdown_options = df_main.columns[3:].tolist()

def generate_color_steps(df, selected_column, n_steps=5):
    min_val = df[selected_column].min()
    max_val = df[selected_column].max()
    step_size = (max_val - min_val) / n_steps
    steps = [min_val + i * step_size for i in range(n_steps)]
    color_scale = px.colors.sequential.Reds
    return {step: color_scale[i] for i, step in enumerate(steps)}

def get_latest_figure(selected_column, initial_year=None):
    df = df_main[df_main[selected_column].notnull()]
    df = df[~df['Entity'].isin(['World', 'Continent'])]

    # Use the provided initial_year or determine the max year
    year_to_display = initial_year if initial_year else df["Year"].max()

    color_map = generate_color_steps(df, selected_column)

    fig = px.choropleth(df[df["Year"] == year_to_display], 
                        locations="Code",
                        color=selected_column,
                        hover_name="Entity",
                        color_discrete_map=color_map,
                        labels={selected_column: selected_column},
                        title=f'Choropleth Map of {selected_column}')

    fig.update_geos(showframe=False, showcoastlines=False, projection_type='equirectangular')
    return fig

app.layout = html.Div([
    dcc.Dropdown(
        id='column_dropdown',
        options=[{'label': i, 'value': i} for i in dropdown_options],
        value=dropdown_options[0],
        clearable=False,
        searchable=False
    ),
    dcc.Graph(
        id='choropleth_map', 
        figure=get_latest_figure(dropdown_options[0]),
        style={"height": "70vh", "width": "124.5vh"}
    ),
])

def generate_color_steps(df, selected_column, n_steps=5):
    min_val = df[selected_column].min()
    max_val = df[selected_column].max()
    step_size = (max_val - min_val) / n_steps
    steps = [min_val + i * step_size for i in range(n_steps)]
    color_scale = px.colors.sequential.Reds
    return {step: color_scale[i] for i, step in enumerate(steps)}

@app.callback(
    Output('choropleth_map', 'figure'),
    [Input('column_dropdown', 'value')]
)
def update_figure(selected_column):
    df = df_main[df_main[selected_column].notnull()]
    df = df[~df['Entity'].isin(['World', 'Continent'])]

    color_map = generate_color_steps(df, selected_column)
    
    fig = px.choropleth(df, 
                        locations="Code",
                        color=selected_column,
                        hover_name="Entity",
                        animation_frame="Year",
                        color_discrete_map=color_map,
                        labels={selected_column: selected_column},
                        title=f'Choropleth Map of {selected_column}')

    fig.update_geos(showframe=False, showcoastlines=False, projection_type='equirectangular')

    # Erhalten Sie den Namen des letzten Frames (maximales Jahr)
    last_frame_name = fig.frames[-1].name

    # Erstellen Sie die Schritte für den Slider
    slider_steps = []
    for frame in fig.frames:
        step = dict(args=[[frame.name], 
                          {"frame": {"duration": 500, "redraw": True},
                           "mode": "immediate",
                           "transition": {"duration": 0}}],
                    label=frame.name,
                    method="animate")
        slider_steps.append(step)

    # Update Slider-Einstellungen
    fig.layout.update(sliders=[{
        'active': len(fig.frames) - 1,  # Setzen Sie den aktiven Schritt auf den letzten Schritt
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 500, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': slider_steps  # Zuweisung der Schritte zum Slider
    }])

    fig.layout.sliders[0].steps[-1]["args"][1]["frame"]["duration"] = 0
    fig.layout.sliders[0].steps[-1]["args"][1]["transition"]["duration"] = 0

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

